In [1]:
import numpy as np
import xarray as xr

import datetime as dt
import os
import pandas as pd

import matplotlib.pyplot as plt

import altair as alt
alt.data_transformers.enable('json')

from sublimpy import utils
import glob
import pytz
import re
from scipy.signal import welch, csd
from scipy.stats import chi2

import sys
sys.path.append('../../paper2/')
from process_fast_data.fast_data_calculate_spectra_nomrd import calculate_mrd_for_df, fast_data_files_to_dataframe
from sublimpy import extrautils

In [2]:
# DATA_DIR = "/storage/elilouis/sublimationofsnow/"
DATA_DIR = "/Users/elischwat/Development/data/sublimationofsnow/"

In [3]:
file_0517 = sorted(glob.glob(os.path.join(
    DATA_DIR, "sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230517*.nc"
)))[18]
file_0517

'/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230517_18.nc'

In [4]:
file_0518 = sorted(glob.glob(os.path.join(
    DATA_DIR, "sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230518*.nc"
)))[18]
file_0518

'/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230518_18.nc'

In [5]:
fast_df_0517 = fast_data_files_to_dataframe( [file_0517], rotation='none')
fast_df_0517 = utils.modify_df_timezone(fast_df_0517, 'UTC', 'US/Mountain').set_index('time')

fast_df_0518 = fast_data_files_to_dataframe( [file_0518], rotation='none')
fast_df_0518 = utils.modify_df_timezone(fast_df_0518, 'UTC', 'US/Mountain').set_index('time')

Apply double rotation

In [6]:
def double_rotation(df, u_col, v_col, w_col):
    # FIRST ROTATION
    mean_u = df[u_col].mean()
    mean_v = df[v_col].mean()
    theta = np.arctan2(mean_v, mean_u)
    adj_u = df[u_col]*np.cos(theta) + df[v_col]*np.sin(theta)
    adj_v = -df[u_col]*np.sin(theta) + df[v_col]*np.cos(theta)
    df[u_col] = adj_u
    df[v_col] = adj_v
    print(
        'Means after 1st rotation:',
        df[u_col].mean(),
        df[v_col].mean(),
        df[w_col].mean(),
    )

    # SECOND ROTATION
    mean_u = df[u_col].mean()
    mean_w = df[w_col].mean()
    phi = np.arctan2(mean_w, mean_u)
    adj_u = df[u_col]*np.cos(phi) + df[w_col]*np.sin(phi)
    adj_w = - df[u_col]*np.sin(phi) + df[w_col]*np.cos(phi)
    df[u_col] = adj_u
    df[w_col] = adj_w
    print(
        'Means after 2nd rotation:',
        df[u_col].mean(),
        df[v_col].mean(),
        df[w_col].mean(),
    )
    return df

In [7]:
fast_df_0517 = double_rotation(fast_df_0517, 'u_2m_c', 'v_2m_c', 'w_2m_c')
fast_df_0517['u_2m_c'] = fast_df_0517['u_2m_c'].interpolate()
fast_df_0517['v_2m_c'] = fast_df_0517['v_2m_c'].interpolate()
fast_df_0517['w_2m_c'] = fast_df_0517['w_2m_c'].interpolate()
fast_df_0517['tc_2m_c'] = fast_df_0517['tc_2m_c'].interpolate()

Means after 1st rotation: 3.045368 8.138021e-08 0.018009925
Means after 2nd rotation: 3.0454214 8.138021e-08 4.2385526e-10


In [8]:
fast_df_0518 = double_rotation(fast_df_0518, 'u_2m_c', 'v_2m_c', 'w_2m_c')
fast_df_0518['u_2m_c'] = fast_df_0518['u_2m_c'].interpolate()
fast_df_0518['v_2m_c'] = fast_df_0518['v_2m_c'].interpolate()
fast_df_0518['w_2m_c'] = fast_df_0518['w_2m_c'].interpolate()
fast_df_0518['tc_2m_c'] = fast_df_0518['tc_2m_c'].interpolate()

Means after 1st rotation: 7.0282445 3.797743e-07 -0.0019361756
Means after 2nd rotation: 7.028244 3.797743e-07 1.9073487e-09


In [9]:
mrd_uw_sos_0517 = calculate_mrd_for_df(
    fast_df_0517[['u_2m_c', 'v_2m_c', 'w_2m_c', 'tc_2m_c']].reset_index(), 
    'u_2m_c', 'w_2m_c', 
    shift=2400, # 2 minute sliding window
    parallelism=20,
    M=15, # 27.31 minute long calculations
)
mrd_vw_sos_0517 = calculate_mrd_for_df(
    fast_df_0517[['u_2m_c', 'v_2m_c', 'w_2m_c', 'tc_2m_c']].reset_index(), 
    'v_2m_c', 'w_2m_c', 
    shift=2400, # 2 minute sliding window
    parallelism=20,
    M=15, # 27.31 minute long calculations
)
mrd_uv_sos_0517 = calculate_mrd_for_df(
    fast_df_0517[['u_2m_c', 'v_2m_c', 'w_2m_c', 'tc_2m_c']].reset_index(), 
    'u_2m_c', 'v_2m_c', 
    shift=2400, # 2 minute sliding window
    parallelism=20,
    M=15, # 27.31 minute long calculations
)
mrd_wtc_sos_0517 = calculate_mrd_for_df(
    fast_df_0517[['u_2m_c', 'v_2m_c', 'w_2m_c', 'tc_2m_c']].reset_index(), 
    'w_2m_c', 'tc_2m_c', 
    shift=2400, # 2 minute sliding window
    parallelism=20,
    M=15, # 27.31 minute long calculations
)


mrd_uu_sos_0517 = calculate_mrd_for_df(
    fast_df_0517[['u_2m_c', 'v_2m_c', 'w_2m_c', 'tc_2m_c']].reset_index(), 
    'u_2m_c', 'u_2m_c', 
    shift=2400, # 2 minute sliding window
    parallelism=20,
    M=15, # 27.31 minute long calculations
)
mrd_vv_sos_0517 = calculate_mrd_for_df(
    fast_df_0517[['u_2m_c', 'v_2m_c', 'w_2m_c', 'tc_2m_c']].reset_index(), 
    'v_2m_c', 'v_2m_c', 
    shift=2400, # 2 minute sliding window
    parallelism=20,
    M=15, # 27.31 minute long calculations
)
mrd_ww_sos_0517 = calculate_mrd_for_df(
    fast_df_0517[['u_2m_c', 'v_2m_c', 'w_2m_c', 'tc_2m_c']].reset_index(), 
    'w_2m_c', 'w_2m_c', 
    shift=2400, # 2 minute sliding window
    parallelism=20,
    M=15, # 27.31 minute long calculations
)

Got data of length 72000. using M = 15
Timestep of fast data is: 50.0 ms
Dataset of length 72000, with M = 15, permits 16 iterations.


100%|██████████| 16/16 [00:00<00:00, 1002.51it/s]


Got data of length 72000. using M = 15
Timestep of fast data is: 50.0 ms
Dataset of length 72000, with M = 15, permits 16 iterations.


100%|██████████| 16/16 [00:00<00:00, 19611.01it/s]


Got data of length 72000. using M = 15
Timestep of fast data is: 50.0 ms
Dataset of length 72000, with M = 15, permits 16 iterations.


...

100%|██████████| 16/16 [00:00<00:00, 31126.56it/s]


Got data of length 72000. using M = 15
Timestep of fast data is: 50.0 ms
Dataset of length 72000, with M = 15, permits 16 iterations.


100%|██████████| 16/16 [00:00<00:00, 25439.30it/s]


Got data of length 72000. using M = 15
Timestep of fast data is: 50.0 ms
Dataset of length 72000, with M = 15, permits 16 iterations.


100%|██████████| 16/16 [00:00<00:00, 25477.93it/s]


In [ ]:
# mrd_uw_sos_0518 = calculate_mrd_for_df(
#     fast_df_0518[['u_2m_c', 'v_2m_c', 'w_2m_c', 'tc_2m_c']].reset_index(), 
#     'u_2m_c', 'w_2m_c', 
#     shift=2400, # 2 minute sliding window
#     parallelism=20,
#     M=15, # 27.31 minute long calculations
# )
# mrd_vw_sos_0518 = calculate_mrd_for_df(
#     fast_df_0518[['u_2m_c', 'v_2m_c', 'w_2m_c', 'tc_2m_c']].reset_index(), 
#     'v_2m_c', 'w_2m_c', 
#     shift=2400, # 2 minute sliding window
#     parallelism=20,
#     M=15, # 27.31 minute long calculations
# )
# mrd_wtc_sos_0518 = calculate_mrd_for_df(
#     fast_df_0518[['u_2m_c', 'v_2m_c', 'w_2m_c', 'tc_2m_c']].reset_index(), 
#     'w_2m_c', 'tc_2m_c', 
#     shift=2400, # 2 minute sliding window
#     parallelism=20,
#     M=15, # 27.31 minute long calculations
# )

In [10]:
def chart(src):
    return alt.Chart(src).mark_line().encode(
        alt.X('tau:Q').scale(type='log').title('𝜏 (s)'),
        alt.Y('mean(Co)').title('Cospectral power'),
        alt.Color('type:N').title(None),
    ).properties(width=150, height=100)
def chart_errorband(src):
    return alt.Chart(src).mark_errorband(extent='iqr').encode(
        alt.X('tau:Q').scale(type='log').title('𝜏 (s)'),
        alt.Y('Co:Q'),
    ).properties(width=150, height=100)


In [11]:
# Create a DataFrame with a single x value for the vertical line
data = pd.DataFrame({'x': [300, 900, 1800]})

# Create the vertical line chart
vertical_line = alt.Chart(data).mark_rule(color='red').encode(
    x='x:Q'  # Quantitative x-axis
)

# data_text = pd.DataFrame({'x': ['300s', '1800s']})

text = alt.Chart(data).mark_text(angle=90, size=8, color='red', align='left', dy=-4).encode(
    x='x:Q',
    text='x:Q'  # Display the x value as text
)
annotations = vertical_line + text
annotations

alt.LayerChart(...)

In [15]:
(
    annotations + 
    chart(mrd_wtc_sos_0517.groupby('tau')[['Co']].mean().reset_index().assign(type='MRD'))+
    chart(mrd_wtc_sos_0517.groupby('tau')[['Co']].mean().cumsum().reset_index().assign(type='ogive'))
).properties(title="w'T'") | (
    annotations + 
    chart(mrd_uw_sos_0517.groupby('tau')[['Co']].mean().reset_index().assign(type='MRD'))+
    chart(mrd_uw_sos_0517.groupby('tau')[['Co']].mean().cumsum().reset_index().assign(type='ogive'))
).properties(title="u'w'") | (
    annotations + 
    chart(mrd_vw_sos_0517.groupby('tau')[['Co']].mean().reset_index().assign(type='MRD'))+
    chart(mrd_vw_sos_0517.groupby('tau')[['Co']].mean().cumsum().reset_index().assign(type='ogive'))
).properties(title="v'w'")  | (
    annotations + 
    chart(mrd_uv_sos_0517.groupby('tau')[['Co']].mean().reset_index().assign(type='MRD'))+
    chart(mrd_uv_sos_0517.groupby('tau')[['Co']].mean().cumsum().reset_index().assign(type='ogive'))
).properties(title="u'v'") 

alt.HConcatChart(...)

In [16]:
(
    annotations + 
    chart(mrd_uu_sos_0517.groupby('tau')[['Co']].mean().reset_index().assign(type='MRD'))+
    chart(mrd_uu_sos_0517.groupby('tau')[['Co']].mean().cumsum().reset_index().assign(type='ogive'))
).properties(title="u'u'") | (
    annotations + 
    chart(mrd_vv_sos_0517.groupby('tau')[['Co']].mean().reset_index().assign(type='MRD'))+
    chart(mrd_vv_sos_0517.groupby('tau')[['Co']].mean().cumsum().reset_index().assign(type='ogive'))
).properties(title="v'v'") | (
    annotations + 
    chart(mrd_ww_sos_0517.groupby('tau')[['Co']].mean().reset_index().assign(type='MRD'))+
    chart(mrd_ww_sos_0517.groupby('tau')[['Co']].mean().cumsum().reset_index().assign(type='ogive'))
).properties(title="w'w'") 

alt.HConcatChart(...)

In [13]:
mrds_0517 = (
    chart_errorband(mrd_wtc_sos_0517.reset_index()) +\
    chart(mrd_wtc_sos_0517.groupby('tau')[['Co']].mean().reset_index())
).properties(title="w'T'") | (
    chart_errorband(mrd_uw_sos_0517.reset_index()) +\
    chart(mrd_uw_sos_0517.groupby('tau')[['Co']].mean().reset_index())
).properties(title="u'w'") | (
    chart_errorband(mrd_vw_sos_0517.reset_index()) +\
    chart(mrd_vw_sos_0517.groupby('tau')[['Co']].mean().reset_index())
).properties(title="v'w'")

In [14]:
mrds_0518 = (
    chart_errorband(mrd_wtc_sos_0518.reset_index()) +\
    chart(mrd_wtc_sos_0518.groupby('tau')[['Co']].mean().reset_index())
).properties(title="w'T'") | (
    chart_errorband(mrd_uw_sos_0518.reset_index()) +\
    chart(mrd_uw_sos_0518.groupby('tau')[['Co']].mean().reset_index())
).properties(title="u'w'") | (
    chart_errorband(mrd_vw_sos_0518.reset_index()) +\
    chart(mrd_vw_sos_0518.groupby('tau')[['Co']].mean().reset_index())
).properties(title="v'w'")

NameError: name 'mrd_wtc_sos_0518' is not defined

In [ ]:
mrds_0517 & mrds_0518